In [76]:
'''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

'\nAuthor information:\nJoey R. Muffoletto\nUniversity of Texas at Austin\nAutonomous Systems Group\njrmuff@utexas.edu\n'

In [77]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [78]:
NUM_VERTIHUBS = 12 #dependent on trip_data
FLIGHT_SPEED = 60 #m/s
ADDITIONAL_VERTIHUBS = True
NUM_ADDITIONAL_VERTIHUBS = 4
if not ADDITIONAL_VERTIHUBS:
    NUM_ADDITIONAL_VERTIHUBS = 0
MINUTES_OF_DATA = 1


In [79]:
'''
Trip data formatting
'''
path = 'data/Realtime/OpsLimits/trips_' + str(NUM_VERTIHUBS + NUM_ADDITIONAL_VERTIHUBS) + '_minutes-' + str(MINUTES_OF_DATA)
if ADDITIONAL_VERTIHUBS:
    path += '_additional_vhubs_' + str(NUM_ADDITIONAL_VERTIHUBS)
path += '.csv'
trip_data = pd.read_csv(path)
trip_data.drop(trip_data[trip_data.OriginVertihub == trip_data.DestinationVertihub].index, inplace=True) #drop trips where the origin and destination are the same vertiport, reduced from 4801 trips to 1692 with 10 vertiports
trip_data

,Time,OriginVertihub,OriginVertiport,DestinationVertihub,DestinationVertiport
0,0,11,38,5,128
1,60,1,137,11,83
2,60,5,119,11,38
3,60,0,124,9,49
4,0,5,67,6,88
5,0,5,128,13,38
6,60,8,59,2,87
7,0,11,39,8,57
8,0,5,140,0,25
9,0,9,100,3,58


In [80]:
path = 'data/Realtime/OpsLimits/vertiports_' + str(NUM_VERTIHUBS + NUM_ADDITIONAL_VERTIHUBS)
if ADDITIONAL_VERTIHUBS:
    path += '_additional_vhubs_' + str(NUM_ADDITIONAL_VERTIHUBS)
path += '.csv'
vertiports_data = pd.read_csv(path)
vertiports_data

,vertiport,long,lat,vertihub_0,vertihub_1
0,0,-87.867935,42.113023,5,12
1,1,-87.983678,41.755379,1,14
2,2,-88.996864,42.431453,0,-1
3,3,-87.008659,41.319802,2,-1
4,4,-87.706935,41.735222,11,13
5,5,-88.370978,41.721233,1,14
6,6,-88.705400,42.141602,6,15
7,7,-88.160202,42.189399,5,12
8,8,-88.940794,41.460231,4,-1
9,9,-88.095275,42.762673,3,-1


In [81]:
'''
vertiport statistics
'''
vertiports_per_vertihub = [0 for i in range(NUM_VERTIHUBS)]
for index, vertiport in vertiports_data.iterrows():
    vertiports_per_vertihub[int(vertiport['vertihub_0'])] += 1
assert(sum(vertiports_per_vertihub) == vertiports_data.shape[0])
vertiports_per_vertihub



[12, 16, 11, 15, 10, 23, 15, 9, 8, 11, 8, 22]

In [82]:
from math import cos, asin, sqrt, pi

# def dist(x1, y1, x2, y2):
#     return math.sqrt((y2-y1)**(y2-y1) + (x2-x1)**(x2-x1))
def ll_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...   
    

In [83]:
'''
trip statistics
'''

arrivals_per_vertihub = [[] for i in range(NUM_VERTIHUBS + NUM_ADDITIONAL_VERTIHUBS)]
flight_popularity = dict()
# get the trip arrival time per vertihub
for index, trip in trip_data.iterrows():
    #calculate time of arrival (begin at trip.time), calculated between vertiports
    origin_port = vertiports_data.iloc[trip['OriginVertiport']]
    destination_port = vertiports_data.iloc[trip['DestinationVertiport']]
    if((origin_port['vertiport'],destination_port['vertiport']) not in flight_popularity.keys()):
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] = 1
    else:
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] += 1
    #trip distance in kilometers
    dist = ll_distance(origin_port['lat'], origin_port['long'], destination_port['lat'], destination_port['long'])
    travel_time = (dist*1000)/FLIGHT_SPEED # (km * 1000)/(m/s) -> seconds
    arrival_time = trip['Time'] + travel_time #takeoff time + travel_time = arrival time
    #add arrival time to vertihub
    arrivals_per_vertihub[trip['DestinationVertihub']].append(arrival_time)

# for each list of vertihub, calculate the minimum, average, and maximum distance between arrivals
differences_per_vertihub = []
# simulated_arrivals
for arrivals_list in arrivals_per_vertihub:
    sorted_arrivals = sorted(arrivals_list)
    differences = []
    if(len(sorted_arrivals) <= 1):
        continue
    for index in range(len(sorted_arrivals)-1):
        differences.append(sorted_arrivals[index+1] - sorted_arrivals[index])
    # print(sorted_arrivals)
    differences_per_vertihub.append(sorted(differences))

for index, differences in enumerate(differences_per_vertihub):
    print('vertihub ' + str(index) + ' has minimum arrival difference of ' + str(differences[0]))
    print('vertihub ' + str(index) + ' has average arrival difference of ' + str(sum(differences)/len(differences)))
    print('vertihub ' + str(index) + ' has maximum arrival difference of ' + str(differences[len(differences)-1]))
    print('vertihub ' + str(index) + ' has ' + str(len(differences)) + ' trips')
    print('')
   

formatted_flight_popularity = [(flight_popularity[key], key) for key in flight_popularity.keys()]
print('Most popular flight path ' + str((sorted(formatted_flight_popularity))[len(formatted_flight_popularity)-1]))
print('Total flight paths ' + str(len(formatted_flight_popularity)))
avg = sum([num for num, thing in formatted_flight_popularity])/len(formatted_flight_popularity)
print('Mean requests per flight path ' + str(avg))
formatted_vertihub_popularity = [(len(differences_per_vertihub[i]), i) for i in range(len(differences_per_vertihub))]

print(sorted(formatted_vertihub_popularity))
    
    

vertihub 0 has minimum arrival difference of 0.0
vertihub 0 has average arrival difference of 27.964722731461208
vertihub 0 has maximum arrival difference of 368.33740404510854
vertihub 0 has 67 trips

vertihub 1 has minimum arrival difference of 0.0
vertihub 1 has average arrival difference of 2.769103457564265
vertihub 1 has maximum arrival difference of 80.70217307332098
vertihub 1 has 305 trips

vertihub 2 has minimum arrival difference of 0.0
vertihub 2 has average arrival difference of 5.468590169910053
vertihub 2 has maximum arrival difference of 52.44847756529782
vertihub 2 has 149 trips

vertihub 3 has minimum arrival difference of 0.0
vertihub 3 has average arrival difference of 8.32450869031714
vertihub 3 has maximum arrival difference of 223.01719208653503
vertihub 3 has 222 trips

vertihub 4 has minimum arrival difference of 0.0
vertihub 4 has average arrival difference of 44.1769226954013
vertihub 4 has maximum arrival difference of 145.07000678866893
vertihub 4 has 23 tr